In [35]:
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, ZeroPadding2D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical

import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import random
import cv2 as cv
import numpy as np

In [36]:
DATADIR = 'dataset'
dirs = []
training_data = []
width, height = 13, 19

# Looping direktori data training untuk diambil nama karakternya
for char_name in sorted(os.listdir(DATADIR)):
    dirs.append(char_name)

# Looping semua image data training untuk diubah menjadi array
for char_name in dirs:
    path = os.path.join(DATADIR, char_name)
    class_number = dirs.index(char_name)

    for img in tqdm(os.listdir(path)):
        try:
            img_array = cv.imread(os.path.join(path, img), cv.IMREAD_GRAYSCALE)
            new_array = cv.resize(img_array, (width, height))
            training_data.append([new_array, class_number])
        except Exception as e:
            pass

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 392.10it/s]


In [37]:
random.shuffle(training_data)

X = []
Y = []

for features, label in training_data:
  X.append(features)
  Y.append(label)

X = np.array(X).reshape(-1, width, height, 1)

# Tulis ke file pickle
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("Y.pickle", "wb")
pickle.dump(Y, pickle_out)
pickle_out.close()

In [38]:
# Load file pickle
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("Y.pickle", "rb")
Y = pickle.load(pickle_in)
Y = to_categorical(Y)
X = X / 255.0
# Input layer
inputs = Input(shape=(width, height, 1))
conv_layer = ZeroPadding2D(padding=(2, 2))(inputs)
conv_layer = Conv2D(16, (5, 5), strides=(1, 1), activation='relu')(conv_layer)
conv_layer = MaxPooling2D((2, 2))(conv_layer)
conv_layer = Conv2D(32, (3, 3), strides=(1, 1), activation='relu')(conv_layer)
conv_layer = Conv2D(32, (3, 3), strides=(1, 1), activation='relu')(conv_layer)
conv_layer = MaxPooling2D((2, 2))(conv_layer)

flaten = Flatten()(conv_layer)
fc_layer = Dense(256, activation='relu')(flaten)
fc_layer = Dense(64, activation='relu')(fc_layer)

# Output layer
outputs = Dense(len(dirs), activation='softmax')(fc_layer)
adam = Adam(learning_rate=0.0001)
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X, Y, epochs=len(dirs)*2, verbose=1)
model.save('alphabet.model')

Epoch 1/300
66/66 [==============================] - 4s 12ms/step - loss: 4.9635 - accuracy: 0.1004
Epoch 2/300
66/66 [==============================] - 1s 10ms/step - loss: 4.5425 - accuracy: 0.1221
Epoch 3/300
66/66 [==============================] - 1s 13ms/step - loss: 4.1169 - accuracy: 0.1221
Epoch 4/300
66/66 [==============================] - 1s 12ms/step - loss: 3.9288 - accuracy: 0.1221
Epoch 5/300
66/66 [==============================] - 1s 11ms/step - loss: 3.8075 - accuracy: 0.1221
Epoch 6/300
66/66 [==============================] - 1s 12ms/step - loss: 3.7430 - accuracy: 0.1221
Epoch 7/300
66/66 [==============================] - 1s 11ms/step - loss: 3.7152 - accuracy: 0.1221
Epoch 8/300
66/66 [==============================] - 1s 11ms/step - loss: 3.6950 - accuracy: 0.1221
Epoch 9/300
66/66 [==============================] - 1s 11ms/step - loss: 3.6811 - accuracy: 0.1221
Epoch 10/300
66/66 [==============================] - 1s 11ms/step - loss: 3.6725 - accuracy: 0.1221

INFO:tensorflow:Assets written to: alphabet.model\assets
